In [ ]:
import tensorflow as tf
import numpy as np
import pickle
import os

Getting the Data
------

In [ ]:
CIFAR_DIR = 'C:/Users/brian/Desktop/TensorFlow/cifar-10-batches-py'

def load_data(filename):
    """ READ DATA FROM BATCH FILES """
    with open(filename, "rb") as file:
        data = pickle.load(file, encoding="bytes")

        d_decoded = {}
        for k, v in data.items():
            d_decoded[k.decode('utf8')] = v

        data = d_decoded

    return(data['data'], data['labels'])

class CifarData:
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_labels = []

        for filename in filenames:
            data, labels = load_data(filename)
            for item, label in zip(data, labels):
                all_data.append(item)
                all_labels.append(label)
                
        self._data = np.vstack(all_data)
        self._data = self._data / 127.5 - 1
        self._labels = np.hstack(all_labels)
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0

        if(self._need_shuffle):
            self.shuffle_data()

    def shuffle_data(self):
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]

    def next_batch(self, batch_size):
        end_indicator = self._indicator + batch_size

        if(end_indicator > self._num_examples):
            if (self._need_shuffle):
                self.shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("no more data")
        
        if (end_indicator > self._num_examples):
            raise Exception("Batch Size Too Large")

        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]

        self._indicator = end_indicator
        return(batch_data, batch_labels)

In [ ]:
train_filenames = [os.path.join(CIFAR_DIR,'data_batch_%d' % i) for i in range(1, 6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)

Getting the Model
------

In [ ]:
x = tf.placeholder(tf.float32, [None, 3072]) 
y = tf.placeholder(tf.int64, [None])

x_image = tf.reshape(x, [-1, 3, 32, 32])
x_image = tf.transpose(x_image, perm=[0, 2, 3, 1])

### Model With 3 Double-Conv Layers Followed by 3 Max-Poolings

In [ ]:

conv1_1 = tf.layers.conv2d(x_image, 32, (3, 3), padding="same", activation=tf.nn.relu, name="conv1_1")

conv1_2 = tf.layers.conv2d(conv1_1, 32, (3, 3), padding="same", activation=tf.nn.relu, name="conv1_2")
pooling1 = tf.layers.max_pooling2d(conv1_2, (2,2), (2,2), name="pooling1")

conv2_1 = tf.layers.conv2d(pooling1, 32, (3, 3), padding="same", activation=tf.nn.relu, name="conv2_1")
conv2_2 = tf.layers.conv2d(conv2_1, 32, (3, 3), padding="same", activation=tf.nn.relu, name="conv2_2")
pooling2 = tf.layers.max_pooling2d(conv2_2, (2,2), (2,2), name="pooling2")

conv3_1 = tf.layers.conv2d(pooling2, 32, (3, 3), padding="same", activation=tf.nn.relu, name="conv3_1")
conv3_2 = tf.layers.conv2d(conv3_1, 32, (3, 3), padding="same", activation=tf.nn.relu, name="conv3_2")
pooling3 = tf.layers.max_pooling2d(conv3_2, (2,2), (2,2), name="pooling3")

flatten = tf.layers.flatten(pooling3)

y_ = tf.layers.dense(flatten, 10)

''' Mean Square Loss
p_y = tf.nn.softmax(y_)

y_one_hot = tf.one_hot(y, 10, dtype=tf.float32)

loss = tf.reduce_mean(tf.square(y_one_hot - p_y))
'''

''' Cross Entropy '''
''' Logits automatically doing softmax for us '''
''' Labels automatically doing one_hot for us '''
''' ylogy_ <- cross entropy calculation function'''

loss = tf.losses.sparse_softmax_cross_entropy(labels = y, logits = y_)

predict = tf.argmax(y_, 1)
correct_prediction = tf.equal(predict, y)

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

init = tf.global_variables_initializer()

batch_size = 20
train_steps = 10000

test_steps = 200

with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, accu_val, _ = sess.run(
            [loss, accuracy, train_op], 
            feed_dict={
                x: batch_data, 
                y: batch_labels})

        if (i+1) % 100 == 0:
            print('[Train] Step: %d, loss: %4.5f, acc: %4.5f' \
                % (i+1, loss_val, accu_val))

        if (i+1) % 1000 == 0:
            test_data = CifarData(test_filenames, False)
            all_test_acc_val = [] 

            for j in range(test_steps):
                test_batch_data, test_batch_labels = test_data.next_batch(batch_size)
                test_acc_val = sess.run(
                    [accuracy],
                    feed_dict = {
                        x: test_batch_data, 
                        y: test_batch_labels
                })

                all_test_acc_val.append(test_acc_val)   

            test_acc = np.mean(all_test_acc_val)

            print('[Test] Step: %d, Overall Acc: %4.5f' % (i+1, test_acc))
            